In [1]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Embedding,
    LSTM,
    Dense,
    Bidirectional,
    Attention,
    TextVectorization,
)
from tensorflow.keras.models import Model
from tensorflow import keras
import numpy as np
import pandas as pd

2025-03-18 22:37:23.481815: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-18 22:37:23.491792: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742317643.503907    3904 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742317643.507367    3904 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-18 22:37:23.520770: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
import string
import re

In [3]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")


vocab_size = 15000
sequence_length = 20

In [4]:
vocab_size = 20000
sequence_length = 20

In [5]:
port = TextVectorization(
    max_tokens=vocab_size,
    output_mode="tf_idf",
    # output_sequence_length=sequence_length ,
    standardize=custom_standardization,
)

span = TextVectorization(
    max_tokens=vocab_size,
    output_mode="tf_idf",
    # output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

eng = TextVectorization(
    max_tokens=vocab_size,
    output_mode="tf_idf",
    # output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

I0000 00:00:1742317650.146007    3904 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4309 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 6GB Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [6]:
port_eng = pd.read_pickle("./train_en_pt/train_0.pkl")

In [7]:
eng_span = pd.read_pickle("./train_en_es/train_0.pkl")

In [8]:
def add_token(token,sent):
    return token+" "+ sent

In [9]:
def add_start_end(sent):
    return "[start] " + sent + " [end]"

In [10]:
port_eng["port_sent"] = port_eng["port_sent"].apply(lambda x:add_token("<EN2>",x))
port_eng["en_sent"] = port_eng["en_sent"].apply(lambda x:add_start_end(x))

In [11]:
eng_span["en_sent"] = eng_span["en_sent"].apply(lambda x: add_token("<SP2>", x))
eng_span["span_sent"] = eng_span["span_sent"].apply(lambda x: add_start_end(x))

In [12]:
print(port_eng["port_sent"][0])
print(eng_span["en_sent"][0])

<EN2> Doze mais vinte e um são trinta e três. Treze mais trinta e um são quarenta e quatro.
<SP2> You could also hire a car.


In [13]:
print(port_eng["en_sent"][0])
print(eng_span["span_sent"][0])

[start] Twelve plus twenty-one is thirty-three. Thirteen plus thirty-one is forty-four. [end]
[start] Podría también alquilar un coche. [end]


In [14]:
port.adapt(port_eng["port_sent"])

In [15]:
eng_vocab = pd.concat([eng_span["en_sent"], eng_span["span_sent"]])

In [ ]:
eng_vocab

0                          <SP2> You could also hire a car.
1                   <SP2> When should we make reservations?
2                                  <SP2> You're too normal.
3                           <SP2> I hope you like the wine.
4         <SP2> My small bladder has me constantly runni...
                                ...                        
270672                 [start] Hace hoy muy buen día. [end]
270673                   [start] Apareció en mi casa. [end]
270674        [start] El perro está royendo un hueso. [end]
270675                   [start] Bienvenida, Meruert. [end]
270676    [start] Llamé a Tom para darle las gracias. [end]
Length: 541354, dtype: object

In [16]:
eng.adapt(eng_vocab)

In [17]:
# eng.adapt(eng_span["en_sent"])
span.adapt(eng_span[["span_sent"]])

In [30]:
len(span("[start] jj hola [end]"))

18269

In [ ]:
port("<EN2> hello is it mw [end]")

<tf.Tensor: shape=(20,), dtype=int64, numpy=array([2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>

In [ ]:
eng("<SP2> hello is donkey [end]")

<tf.Tensor: shape=(21,), dtype=int64, numpy=
array([   2, 2351,   16, 8904,    4,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0])>

In [18]:
eng_d = eng.get_vocabulary()
eng_look_up = dict(zip(range(len(eng_d)),eng_d))
print(eng_look_up[2])
print(eng_look_up[4])

[start]
a


In [19]:
def format_dataset(eng_, port_):
    port_ = port(port_)
    eng_ = eng(eng_)
    return (
        {
            "portugese":port_ ,
            "english": eng_[:, :-1],
        },
        eng_[:, 1:],
    )


def make_dataset(pairs):
    eng_texts, spa_texts = pairs
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(32)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

In [ ]:
train_ds = make_dataset([port_eng["port_sent"], port_eng["en_sent"]])

In [ ]:

for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['portugese'].shape}")
    print(f"inputs['spanish'].shape: {inputs['english'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (32, 20)
inputs['spanish'].shape: (32, 20)
targets.shape: (32, 20)
inputs['english'].shape: (32, 20)
inputs['spanish'].shape: (32, 20)
targets.shape: (32, 20)
inputs['english'].shape: (32, 20)
inputs['spanish'].shape: (32, 20)
targets.shape: (32, 20)
inputs['english'].shape: (32, 20)
inputs['spanish'].shape: (32, 20)
targets.shape: (32, 20)
inputs['english'].shape: (32, 20)
inputs['spanish'].shape: (32, 20)
targets.shape: (32, 20)
inputs['english'].shape: (32, 20)
inputs['spanish'].shape: (32, 20)
targets.shape: (32, 20)
inputs['english'].shape: (32, 20)
inputs['spanish'].shape: (32, 20)
targets.shape: (32, 20)
inputs['english'].shape: (32, 20)
inputs['spanish'].shape: (32, 20)
targets.shape: (32, 20)
inputs['english'].shape: (32, 20)
inputs['spanish'].shape: (32, 20)
targets.shape: (32, 20)
inputs['english'].shape: (32, 20)
inputs['spanish'].shape: (32, 20)
targets.shape: (32, 20)
inputs['english'].shape: (32, 20)
inputs['spanish'].shape: (32, 20)
targets.shap

2025-03-15 12:49:26.343256: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, lstm_units):
        super(Encoder, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.lstm = Bidirectional(
            LSTM(lstm_units, return_sequences=True, return_state=True)
        )

    def call(self, inputs):
        x = self.embedding(inputs)
        output, forward_h, forward_c, backward_h, backward_c = self.lstm(x)
        # Combine forward and backward states
        state_h = tf.concat([forward_h, backward_h], axis=-1)
        state_c = tf.concat([forward_c, backward_c], axis=-1)
        return output, state_h, state_c  # Return sequence outputs and final state

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)  # Encoder output transformation
        self.W2 = Dense(units)  # Decoder hidden state transformation
        self.V = Dense(1)  # Attention score

    def call(self, encoder_output, decoder_hidden):
        # Expand decoder hidden state for broadcasting
        decoder_hidden_with_time_axis = tf.expand_dims(decoder_hidden, 1)
        # Calculate attention scores
        score = self.V(
            tf.nn.tanh(self.W1(encoder_output) + self.W2(decoder_hidden_with_time_axis))
        )
        attention_weights = tf.nn.softmax(score, axis=1)  # Softmax across time axis
        context_vector = attention_weights * encoder_output
        context_vector = tf.reduce_sum(context_vector, axis=1)  # Sum across time axis
        return context_vector, attention_weights

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, lstm_units):
        super(Decoder, self).__init__()
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.lstm = Bidirectional(
            LSTM(lstm_units, return_sequences=True)
        )
        self.fc = Dense(vocab_size, activation="softmax")
        self.attention = BahdanauAttention(lstm_units)

    def call(self, inputs, encoder_output, hidden_state):
        x = self.embedding(inputs)  # Shape: (batch_size, seq_len, embedding_dim)
        context_vectors = []  # Store attention context for each timestep

        for t in range(20):  # Iterate over time steps
            context_vector, _ = self.attention(encoder_output, hidden_state)
            context_vectors.append(context_vector)

        context_vectors = tf.stack(
            context_vectors, axis=1
        )  # Shape: (batch_size, seq_len, lstm_units)

        x = tf.concat(
            [context_vectors, x], axis=-1
        )  # Concatenate across last axis

        output = self.lstm(x)
        # state_h = tf.concat([forward_h, backward_h], axis=-1)
        # state_c = tf.concat([forward_c, backward_c], axis=-1)

        output = self.fc(output)  # Convert to vocabulary predictions
        return output

In [ ]:
source = keras.Input(shape=(None,), dtype="int64", name="portugese")
encoded_source = Encoder(VOCAB_SIZE,EMBEDDING_DIM,LSTM_UNITS)(source)

In [ ]:
past_target = keras.Input(shape=(None,), dtype="int64", name="english")
out = Decoder(VOCAB_SIZE, EMBEDDING_DIM, LSTM_UNITS)(past_target,*encoded_source[:-1])
seq2seq_rnn = keras.Model([source, past_target], out)

In [2]:
seq2seq_rnn.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

NameError: name 'seq2seq_rnn' is not defined

In [ ]:
seq2seq_rnn.fit(train_ds, epochs=1)

I0000 00:00:1742023194.050724   11225 cuda_dnn.cc:529] Loaded cuDNN version 90300


8459/8459 ━━━━━━━━━━━━━━━━━━━━ 1227s 144ms/step - accuracy: 0.6557 - loss: nan


In [ ]:
class Seq2Seq(Model):
    def __init__(self, vocab_size, embedding_dim, lstm_units):
        super(Seq2Seq, self).__init__()
        self.encoder = Encoder(vocab_size, embedding_dim, lstm_units)
        self.decoder = Decoder(vocab_size, embedding_dim, lstm_units)

    def call(self, inputs):
        encoder_input, decoder_input = inputs["portugese"], inputs["english"]
        print(encoder_input,decoder_input)
        encoder_output, encoder_h, encoder_c = self.encoder(encoder_input)
        decoder_output, _, _ = self.decoder(decoder_input, encoder_output, encoder_h)
        return decoder_output

    def inference(self, inputs):
        encoder_output, encoder_h, encoder_c = self.encoder(inputs)
        decoder_output, _, _ = self.decoder(decoder_input, encoder_output, encoder_h)
        return decoder_output

In [ ]:
EMBEDDING_DIM = 300  # Word embedding dimension
LSTM_UNITS = 512  # LSTM hidden units
VOCAB_SIZE = 10000  # Vocabulary size
MAX_LEN = 20  # Max sequence length

In [ ]:
# Instantiate the model
model = Seq2Seq(VOCAB_SIZE, EMBEDDING_DIM, LSTM_UNITS)

# Compile
model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [ ]:
model.fit(train_ds,
    epochs=10,
)

Epoch 1/10


I0000 00:00:1742017882.252997    5772 cuda_dnn.cc:529] Loaded cuDNN version 90300


8459/8459 ━━━━━━━━━━━━━━━━━━━━ 1224s 144ms/step - accuracy: 0.6554 - loss: nan
Epoch 2/10
 477/8459 ━━━━━━━━━━━━━━━━━━━━ 18:52 142ms/step - accuracy: 0.6564 - loss: nan

KeyboardInterrupt: 

In [ ]:
def decode_sequence(input_sentence):
    tokenized_input_sentence = port([input_sentence])
    decoded_sentence = "[start]"
    for i in range(MAX_LEN):
        tokenized_target_sentence = eng([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            {"portugese":tokenized_input_sentence, "english":tokenized_target_sentence[:,:-1]}
        )
        print(next_token_predictions)
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = eng_look_up[sampled_token_index]
        print(sampled_token)
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence


input_sentence = "<en2> olá, como vai?"
print(input_sentence)
print(decode_sequence(input_sentence))

<en2> olá, como vai?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 496ms/step
[[[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[[[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
[[[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
[[[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]]

1/1 ━━━━━━━━━━━━━━━━━━